## Test Observation Wrapper

In [2]:
from gym.wrappers import TransformObservation
from nclustRL.utils.helper import transform_obs
import nclustenv

In [3]:
env = nclustenv.make('BiclusterEnv-v0')

In [4]:
env2 = TransformObservation(env, transform_obs)
obs = env.reset()['state']
obs_flat = env2.reset()['state']

In [ ]:
obs.ndata

In [ ]:
obs_flat.ndata

In [5]:
import dgl

def transform_obs(n, obs):

    nclusters = n

    state = obs.clone()
    ntypes = state.ntypes

    for n, axis in enumerate(ntypes):
        for i in range(nclusters):
            state.nodes[axis].data[i] = torch.randint(0, 2, (len(state.nodes(axis)),), dtype=torch.bool).to('cuda:0')

    keys = sorted(list(state.nodes[ntypes[0]].data.keys()))
    ndata = {}

    for ntype in ntypes:
        ndata[ntype] = torch.vstack(
            [state.ndata[key][ntype].float() for key in keys]
        ).transpose(0, 1).to('cuda:0')

        state.nodes[ntype].data.clear()
    state.ndata['feat'] = ndata

    return state

## Test Embedings

### Explicit

In [4]:
import dgl
import dgl.nn.pytorch as dglnn
import torch.nn as nn
from torch.nn import functional as F
from nclustRL.utils.helper import pairwise

class HeteroRelu(nn.ReLU):

    def __init__(self, inplace:bool = False):
        super(HeteroRelu, self).__init__(inplace=inplace)

    def forward(self, inputs):
        
        return {k: super(HeteroRelu, self).forward(v) for k, v in inputs.items()}

class GraphSequential(nn.Sequential):

    def __init__(self, *args):
        super(GraphSequential, self).__init__(*args)

    def forward(self, graph, feat, edge_weight=None):
        for module in self:

            if isinstance(module, dglnn.HeteroGraphConv):

                rel_names = zip(module.mods.keys(), graph.canonical_etypes)
                feat = module(
                    g=graph, 
                    inputs=feat, 
                    mod_kwargs={
                        rel: dict(edge_weight=graph.edges[canonical].data[edge_weight]) 
                        for rel, canonical in rel_names})

            else:
                feat = module(inputs=feat)

        return feat


class RGCN(nn.Module):
    def __init__(self, layers, rel_names):
        super().__init__()

        _layers = []

        for in_feats, out_feats in pairwise(layers): 

            _layers.append(dglnn.HeteroGraphConv({
                rel: dglnn.GraphConv(in_feats, out_feats)
                for rel in rel_names}, aggregate='sum'))

            _layers.append(HeteroRelu())

        self._hidden_layers = GraphSequential(*_layers)

            

    def forward(self, graph, feat, edge_weight=None):

        return self._hidden_layers(graph, feat, edge_weight)


class GraphEncoder(nn.Module):
    def __init__(self, n, conv_feats, n_classes, rel_names):
        super().__init__()

        conv_feats.insert(0, n)
        self.rgcn = RGCN(conv_feats, rel_names)

    def forward(self, g):
        h = g.ndata['feat']
        h = self.rgcn(g, h, 'w')
        with g.local_scope():
            g.ndata['h'] = h
            hg = 0
            for ntype in h.keys():
                hg = hg + dgl.mean_nodes(g, 'h', ntype=ntype)
            
            return hg


class HeteroClassifier(nn.Module):
    def __init__(self, n, conv_feats, n_classes, rel_names):
        super().__init__()

        conv_feats.insert(0, n)

        self.rgcn = RGCN(conv_feats, rel_names)
        self.classify = nn.Linear(conv_feats[-1], n_classes)

    def forward(self, g):
        h = g.ndata['feat']
        h = self.rgcn(g, h, 'w')
        print(h['col'])
        with g.local_scope():
            g.ndata['h'] = h
            hg = 0
            for ntype in h.keys():
                hg = hg + dgl.mean_nodes(g, 'h', ntype=ntype)

            return self.classify(hg)

In [5]:
import torch
from torch.nn import functional as F
from tqdm import tqdm
from dgl.dataloading import GraphDataLoader

dgl.seed(5)

def test_embedings(graphs):

    batch_size=1
    shuffle=True
    nclasses = 5
    n = 5

    # dataloader = GraphDataLoader(
    #     base,
    #     batch_size=batch_size,
    #     drop_last=False,
    #     shuffle=shuffle)

    etypes = graphs[0].etypes

    model = HeteroClassifier(n, [n*2], nclasses, etypes)
    model = model.cuda()
    opt = torch.optim.Adam(model.parameters())


    for epoch in range(20):
        with tqdm(graphs, unit="batch") as tepoch:
            for batched_graph in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                # batched_graph = transform_obs(n, batched_graph)
                labels = torch.randint(0, 4, (batch_size,)).to('cuda:0')

                logits = model(batched_graph)
                loss = F.cross_entropy(logits, labels)

                predictions = logits.argmax(dim=1, keepdim=True).squeeze()
                correct = (logits == labels).sum().item()

                opt.zero_grad()
                loss.backward()
                opt.step()

                accuracy = correct / batch_size
                tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)

In [22]:
test_embedings(graphs)

Epoch 0: 100%|███████| 10/10 [00:00<00:00, 104.07batch/s, accuracy=0, loss=1.68]


tensor([[0.0000, 0.5451, 0.0000, 1.2836, 0.0000, 0.0000, 0.0000, 0.0000, 1.7132,
         0.0000],
        [0.0903, 0.7741, 0.0000, 0.7793, 0.0000, 1.3730, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.5813, 0.0000, 3.5214, 0.9626, 0.0000, 1.2651, 1.6148, 2.7820, 0.0000,
         1.0708],
        [0.0000, 0.4411, 0.0000, 0.4172, 0.1244, 0.0000, 0.5478, 0.0000, 0.5815,
         0.0630],
        [0.9421, 0.0000, 1.8847, 0.0000, 0.0793, 0.2830, 0.8410, 2.1577, 0.0000,
         0.6675],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.2982, 0.0000, 0.0000, 0.2387, 0.4370,
         0.0000],
        [0.6426, 0.0000, 1.9349, 2.3578, 0.0000, 0.9353, 2.2055, 0.8362, 0.0000,
         0.9592],
        [0.0000, 0.6578, 0.0000, 0.0000, 0.4252, 0.0000, 0.0000, 0.0000, 0.4283,
         0.0000],
        [0.0000, 0.2671, 0.0000, 0.0000, 0.3966, 0.3184, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.1446, 0.0000, 0.0000, 0.4191, 0.0000, 0.0000, 0.0000, 0.4445,
         0.0000],
        [0

Epoch 1: 100%|███████| 10/10 [00:00<00:00, 107.98batch/s, accuracy=0, loss=1.48]


tensor([[0.0000, 0.5748, 0.0000, 1.3011, 0.0000, 0.0000, 0.0000, 0.0000, 1.7433,
         0.0000],
        [0.0868, 0.8066, 0.0000, 0.7856, 0.0000, 1.3835, 0.0000, 0.0000, 0.0000,
         0.0000],
        [1.5821, 0.0000, 3.4950, 0.9522, 0.0000, 1.2890, 1.5819, 2.8010, 0.0000,
         1.0632],
        [0.0000, 0.4471, 0.0000, 0.4168, 0.1298, 0.0000, 0.5394, 0.0000, 0.5896,
         0.0594],
        [0.9431, 0.0000, 1.8668, 0.0000, 0.0750, 0.2934, 0.8191, 2.1710, 0.0000,
         0.6642],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3069, 0.0000, 0.0000, 0.2423, 0.4420,
         0.0000],
        [0.6439, 0.0000, 1.9162, 2.3539, 0.0000, 0.9449, 2.1766, 0.8506, 0.0000,
         0.9477],
        [0.0000, 0.6748, 0.0000, 0.0000, 0.4399, 0.0000, 0.0000, 0.0000, 0.4450,
         0.0000],
        [0.0000, 0.2897, 0.0000, 0.0000, 0.4171, 0.3299, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.1689, 0.0000, 0.0000, 0.4357, 0.0000, 0.0000, 0.0000, 0.4647,
         0.0000],
        [0

Epoch 2:   0%|                 | 0/10 [00:00<?, ?batch/s, accuracy=0, loss=1.97]

...

tensor([[0.3462, 0.0000, 1.7927, 1.3794, 0.0000, 1.2369, 1.7558, 1.2914, 0.0000,
         0.9769],
        [0.0000, 1.3708, 0.0000, 0.0000, 0.3797, 0.0947, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 1.9674, 0.0000, 0.0000, 0.0000, 0.6143, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 2.1222, 0.0000, 0.0000, 0.2919, 0.0000, 0.0000, 0.0000, 1.4362,
         0.0000],
        [0.0000, 3.2133, 0.0000, 0.0000, 0.0000, 0.1451, 0.0000, 0.0000, 0.7483,
         0.0000],
        [0.0000, 0.0000, 1.0635, 0.5713, 0.0000, 1.2949, 0.0000, 0.0899, 0.0000,
         0.0976],
        [0.0000, 1.3104, 0.0000, 0.0000, 0.5967, 0.0000, 0.0000, 0.0000, 0.6487,
         0.0000],
        [0.0000, 1.8787, 0.0000, 0.1493, 0.1237, 0.0000, 0.8145, 0.0000, 0.1035,
         0.3280],
        [0.5683, 0.4871, 0.3103, 0.1714, 0.0000, 1.1549, 0.2654, 0.0000, 0.0000,
         0.2504],
        [0.0000, 1.4722, 0.0000, 0.6730, 0.0000, 0.4001, 0.0000, 0.0000, 1.1706,
         0.0000],
        [0

Epoch 19:   0%|                | 0/10 [00:00<?, ?batch/s, accuracy=0, loss=1.48]

tensor([[1.3496, 0.0000, 1.2782, 0.1906, 0.3893, 0.1524, 1.3287, 0.9351, 0.0000,
         0.5006],
        [0.0000, 0.6486, 0.0000, 0.0000, 0.1027, 0.1304, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.6764, 0.7752, 0.0000, 0.0000, 0.9048, 0.0432, 0.0000, 0.1167, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0517, 0.0925, 0.2540, 0.0000, 0.3510, 0.1193, 0.3374,
         0.0000],
        [0.0000, 0.7879, 0.0000, 0.0620, 0.0000, 0.0000, 0.0000, 0.0000, 0.8274,
         0.0000],
        [0.0000, 0.0000, 1.1129, 0.0127, 0.0000, 0.3391, 0.0000, 0.6962, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.5389, 0.0000, 0.0000, 0.0671, 0.0000, 0.7373, 0.1101,
         0.0000],
        [0.0000, 0.0000, 0.1766, 0.3128, 0.0000, 0.1659, 0.0000, 0.1347, 0.4134,
         0.0000],
        [0.0000, 2.4093, 0.0000, 0.0000, 0.0000, 0.0832, 0.0000, 0.0000, 0.6847,
         0.0000],
        [0.1875, 0.5106, 0.0000, 0.0000, 0.9717, 0.0000, 0.0000, 0.2261, 0.0000,
         0.0000],
        [0

Epoch 19: 100%|██████| 10/10 [00:00<00:00, 114.33batch/s, accuracy=0, loss=1.66]

tensor([[0.0000, 0.6578, 0.0000, 1.4239, 0.0000, 0.1956, 0.2065, 0.0000, 1.1939,
         0.0629],
        [0.0000, 0.0000, 1.4322, 0.2422, 0.0656, 0.0000, 0.0253, 0.9136, 0.4639,
         0.0000],
        [0.4079, 0.6183, 0.0000, 1.1113, 0.0000, 0.2614, 1.8082, 0.0000, 0.0000,
         0.7027],
        [0.0000, 0.9801, 0.0000, 0.0000, 0.1551, 0.0000, 0.0000, 0.0000, 0.3378,
         0.0000],
        [0.0000, 0.6085, 0.0000, 0.0000, 0.1091, 0.0000, 0.0000, 0.0000, 0.3667,
         0.0000],
        [0.6088, 0.0000, 3.3774, 0.1321, 0.5819, 0.0000, 1.4907, 3.7292, 0.0000,
         0.6818],
        [0.0000, 0.0000, 1.3686, 1.1314, 0.0000, 0.2486, 0.7172, 0.8724, 0.2669,
         0.3284],
        [0.0000, 1.1226, 0.0000, 0.0000, 0.6457, 0.0000, 0.0000, 0.0000, 0.7428,
         0.0000],
        [0.0000, 0.6580, 0.0000, 0.4594, 0.0000, 0.0000, 0.4610, 0.0000, 0.5997,
         0.0778],
        [0.1161, 1.2584, 0.0000, 0.9194, 0.0000, 0.4353, 1.1030, 0.0000, 0.0000,
         0.3936],
        [0

### Built-In

In [8]:
import torch
from torch.nn import functional as F
from tqdm import tqdm
from dgl.dataloading import GraphDataLoader
from nclustRL.models import HeteroClassifier

dgl.seed(5)

def test_embedings(graphs):

    batch_size=1
    shuffle=True
    nclasses = 5
    n = 5

    # dataloader = GraphDataLoader(
    #     base,
    #     batch_size=batch_size,
    #     drop_last=False,
    #     shuffle=shuffle)

    etypes = graphs[0].etypes

    model = HeteroClassifier(n, [n*2], nclasses, etypes)
    model = model.cuda()
    opt = torch.optim.Adam(model.parameters())


    for epoch in range(20):
        with tqdm(graphs, unit="batch") as tepoch:
            for batched_graph in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                # batched_graph = transform_obs(n, batched_graph)
                labels = torch.randint(0, 4, (batch_size,)).to('cuda:0')

                logits = model(batched_graph)
                loss = F.cross_entropy(logits, labels)

                predictions = logits.argmax(dim=1, keepdim=True).squeeze()
                correct = (logits == labels).sum().item()

                opt.zero_grad()
                loss.backward()
                opt.step()

                accuracy = correct / batch_size
                tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)

In [9]:
test_embedings(graphs)

Epoch 19: 100%|██████| 10/10 [00:00<00:00, 172.55batch/s, accuracy=0, loss=1.44]


## Generate Graphs

### Explicit

In [23]:
    import torch as th
    import dgl
    def loader(cls, module=None):

        return getattr(module, cls) if isinstance(cls, str) else cls
    
    def dense_to_dgl(x, device, cuda=0, nclusters=1, clust_init='zeros', duplicate=True):

        # set (u,v)
        clust_init = loader(th, clust_init)

        tensor = th.tensor([[i, j, elem] for i, row in enumerate(x) for j, elem in enumerate(row)]).T

        if duplicate:

            graph_data = {
                ('row', 'elem', 'col'): (tensor[0].int(), tensor[1].int()),
                ('col', 'elem', 'row'): (tensor[1].int().detach().clone(), tensor[2].int().detach().clone()),
                }

            # create graph
            G = dgl.heterograph(graph_data)

            # set weights
            G.edges[('row', 'elem', 'col')].data['w'] = tensor[2].to(th.float64)
            G.edges[('col', 'elem', 'row')].data['w'] = tensor[2].to(th.float64)

        else:

            graph_data = {
                ('row', 'elem', 'col'): (tensor[0].int(), tensor[1].int()),
                }

            # create graph
            G = dgl.heterograph(graph_data)

            # set weights
            # G.edges[('row', 'elem', 'col')].data['w'] = tensor[2].float()

            G.edata['w'] = tensor[2].to(th.float64)

        # set cluster members

        for n, axis in enumerate(['row', 'col']):
            for i in range(nclusters):
                G.nodes[axis].data[i] = th.randint(0, 2, (x.shape[n],), dtype=torch.bool)

        ndata = {}
        ntypes = G.ntypes
        keys = sorted(list(G.nodes[ntypes[0]].data.keys()))

        for ntype in ntypes:
            ndata[ntype] = torch.vstack(
                [G.ndata[key][ntype].to(th.float64) for key in keys]
            ).transpose(0, 1)

            G.nodes[ntype].data.clear()

        G.ndata['feat'] = ndata

        if device == 'gpu':
            G = G.to('cuda:{}'.format(cuda))

        return G

In [24]:
import nclustenv
import torch
env = nclustenv.make('BiclusterEnv-v0', **dict(shape=[[100, 10], [110, 15]], clusters=[5,5]))

graphs_dup = []
graphs2 = []
for i in range(10):
    env.reset()
    X = env.state._generator.X
    graphs_dup.append(dense_to_dgl(X, device='gpu', nclusters=5))
    graphs2.append(dense_to_dgl(X, device='gpu', nclusters=5, duplicate=False))

In [25]:
graphs_dup[0]

Graph(num_nodes={'col': 13, 'row': 102},
      num_edges={('col', 'elem', 'row'): 1326, ('row', 'elem', 'col'): 1326},
      metagraph=[('col', 'row', 'elem'), ('row', 'col', 'elem')])

In [26]:
graphs2[0]

Graph(num_nodes={'col': 13, 'row': 102},
      num_edges={('row', 'elem', 'col'): 1326},
      metagraph=[('row', 'col', 'elem')])

In [27]:
graphs2[0].edges[('row', 'elem', 'col')].data['w']

tensor([ 5.1700,  4.7800,  2.2900,  ...,  4.4100,  8.2300, -9.9300],
       device='cuda:0', dtype=torch.float64)

### Bult-In

In [19]:
import nclustenv
import torch
from gym.wrappers import TransformObservation
from nclustRL.utils.helper import transform_obs

def randint(size, dtype):
    return torch.randint(low=0, high=2, size=[size], dtype=dtype)

env = nclustenv.make('BiclusterEnv-v0', **dict(n=5, shape=[[100, 10], [110, 15]], clusters=[5,5], clust_init=randint))
env = TransformObservation(env, transform_obs)

graphs = [env.reset()['state'] for _ in range(10)]

In [20]:
graphs[0].edges[('row', 'elem', 'col')].data['w']

tensor([-1.3200, -9.7900, -3.0600,  ...,  8.3300, -2.6600,  5.3300],
       device='cuda:0')

In [21]:
graphs[0].ndata['feat']

{'col': tensor([[0., 1., 1., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 1., 0., 1.],
         [1., 0., 1., 1., 1.],
         [1., 0., 0., 0., 0.],
         [1., 0., 1., 1., 1.],
         [0., 1., 0., 1., 0.],
         [1., 1., 0., 0., 1.],
         [1., 1., 1., 1., 0.],
         [1., 1., 0., 0., 1.],
         [1., 1., 1., 1., 0.],
         [1., 1., 1., 0., 1.]], device='cuda:0'),
 'row': tensor([[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [1., 0., 1., 1., 1.],
         [1., 0., 0., 1., 1.],
         [0., 1., 0., 0., 0.],
         [1., 0., 1., 0., 0.],
         [1., 0., 0., 1., 0.],
         [0., 1., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 1., 1.],
         [0., 0., 0., 1., 0.],
         [1., 1., 0., 1., 1.],
         [1., 0., 0., 1., 0.],
         [0., 1., 1., 0., 1.],
         [0., 0., 1., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1.],
         [1., 0., 0., 1., 1.],
      

In [11]:
env.observation_space['state'].n

5

In [12]:
env.action_space[0].n

4

In [25]:
np.prod(env.action_space[1].shape)

12

In [24]:
import numpy as np